In [1]:
# coding: utf-8

# In[1]:


"""
TODO :
test XOR 
    with N layers
    with ReLU & Sigmoid & Softmax
        see gradient gets dead if bias is way too negative
    with Adam vs GradientDescent

"""

# In[2]:

import tensorflow as tf
from tensorflow.python import debug as tf_debug
import numpy as np
import sys

tf.logging.set_verbosity(tf.logging.INFO)

# In[3]:


X = np.array(map(lambda x : map(int,list(x)),("00 01 10 11".split(' '))))
Y = np.reshape(map(int,list("0110")),(4,1))


# In[4]:
from datetime import datetime

In [2]:
class TwoLayerModel():
    def __init__(self,learning_rate=0.01,
		 facW1 = 0.1,		 facW2 = 1,
                 logs_path='/tmp/tensorflow/logs',
                 optimiser='grad'):
        with tf.name_scope('Alpha_'+str(learning_rate)+'_'+optimiser+'_'):
            with tf.name_scope('I/O'):
                self.x_ = tf.placeholder(tf.float32, shape=[4,2], name="x-input")
                self.y_ = tf.placeholder(tf.uint8, shape=[4,1], name="y-input")
           
            with tf.name_scope('Theta'):
                Theta1 = tf.Variable(tf.random_uniform([2,2],-facW1,facW1),name='Theta1')
                Theta2 = tf.Variable(tf.random_uniform([2,1],-facW2,facW2),name='Theta2')
            with tf.name_scope('Bias'):
                Bias1 = tf.Variable(tf.zeros([2]), name="Bias1")
                Bias2 = tf.Variable(tf.zeros([1]), name="Bias2")
            with tf.name_scope('MLP'):
	            hidden_out = tf.sigmoid(tf.matmul(self.x_, Theta1) + Bias1)
	            net_h = tf.matmul(hidden_out, Theta2) + Bias2
	            Hypothesis = tf.sigmoid(net_h,name="Hypothesis")
	            self.y_pred = tf.round(Hypothesis)
	            self.y_float = tf.cast(self.y_,tf.float32)
#	            self.y_pred = tf.cond(Hypothesis > tf.constant([0.5]),lambda: tf.constant([1]),lambda: tf.constant([0]))
		#element wise multiplication will happen here between h and y_-
            self.cost = tf.reduce_mean(-1 * ( (self.y_float * tf.log(Hypothesis)) + ((1 - self.y_float) * tf.log(1.0 - Hypothesis)) ) )

            if (optimiser=='grad'):
                self.train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(self.cost)
            else:
                self.train_step = tf.train.AdamOptimizer(learning_rate).minimize(self.cost)


            self.accuracy = tf.reduce_mean(
                tf.cast(
               tf.equal(self.y_float,self.y_pred),	
                tf.float32)
                ,name="Accuracy")

            self.summary_writer = tf.summary.FileWriter(logs_path+"/"+datetime.now().strftime("%Y%m%d-%H%M%S"), graph=tf.get_default_graph())
            # Create a summary to monitor cost tensor
            tf.summary.scalar("cost", self.cost)
            # Create a summary to monitor accuracy tensor
            tf.summary.scalar("accuracy", self.accuracy)
            # Merge all summaries into a single op - to pass into sess.run
            self.merged_summary_op = tf.summary.merge_all()



        
    
    def train(self,sess,x,y,index):
        _,c,acc,summary = sess.run([self.train_step,self.cost,self.accuracy,self.merged_summary_op],feed_dict={self.x_:x,self.y_:y})
        self.summary_writer.add_summary(summary,index)
        return c,acc

In [3]:
def printbuf(x):
	sys.stdout.write(x)
	sys.stdout.write('\r')
with tf.variable_scope('model1'):
	model1=TwoLayerModel(1,optimiser="grad")  
	
# with tf.variable_scope('model2'):
# 	model2=TwoLayerModel(0.1,optimiser="grad")


with tf.Session() as sess:
	sess.run(tf.global_variables_initializer())
	iters = 1000000
	min_count=100
	count=0
	print('')
	for i in range(iters):
		c,acc =model1.train(sess,X,Y,i)
		
# 		c2,acc2 =model2.train(sess,X,Y,i)
		print('Iteration : ' +str(i) + ' Cost : ' +str(c) + ' Accuracy : ' +str(acc) )
#               + ' \t Iteration : ' +str(i) + ' Cost : ' +str(c2) + ' Accuracy : ' +str(acc2))
		if(acc==1):
			count+=1
			if( count==min_count):
				break	
		else:
			count=0
		#print(model2.train(X,Y))
	print('')

	# In[ ]:
      


Iteration : 0 Cost : 0.712063 Accuracy : 0.5
Iteration : 1 Cost : 0.700645 Accuracy : 0.5
Iteration : 2 Cost : 0.696114 Accuracy : 0.5
Iteration : 3 Cost : 0.694323 Accuracy : 0.5
Iteration : 4 Cost : 0.693614 Accuracy : 0.5
Iteration : 5 Cost : 0.693333 Accuracy : 0.5
Iteration : 6 Cost : 0.693222 Accuracy : 0.5
Iteration : 7 Cost : 0.693177 Accuracy : 0.5
Iteration : 8 Cost : 0.69316 Accuracy : 0.5
Iteration : 9 Cost : 0.693153 Accuracy : 0.25
Iteration : 10 Cost : 0.69315 Accuracy : 0.25
Iteration : 11 Cost : 0.693149 Accuracy : 0.5
Iteration : 12 Cost : 0.693148 Accuracy : 0.5
Iteration : 13 Cost : 0.693148 Accuracy : 0.5
Iteration : 14 Cost : 0.693148 Accuracy : 0.5
Iteration : 15 Cost : 0.693148 Accuracy : 0.5
Iteration : 16 Cost : 0.693148 Accuracy : 0.5
Iteration : 17 Cost : 0.693148 Accuracy : 0.5
Iteration : 18 Cost : 0.693148 Accuracy : 0.5
Iteration : 19 Cost : 0.693148 Accuracy : 0.5
Iteration : 20 Cost : 0.693148 Accuracy : 0.5
Iteration : 21 Cost : 0.693148 Accuracy : 0

Iteration : 307 Cost : 0.693147 Accuracy : 0.5
Iteration : 308 Cost : 0.693147 Accuracy : 0.5
Iteration : 309 Cost : 0.693147 Accuracy : 0.5
Iteration : 310 Cost : 0.693147 Accuracy : 0.5
Iteration : 311 Cost : 0.693147 Accuracy : 0.5
Iteration : 312 Cost : 0.693147 Accuracy : 0.5
Iteration : 313 Cost : 0.693147 Accuracy : 0.5
Iteration : 314 Cost : 0.693147 Accuracy : 0.5
Iteration : 315 Cost : 0.693147 Accuracy : 0.5
Iteration : 316 Cost : 0.693147 Accuracy : 0.5
Iteration : 317 Cost : 0.693147 Accuracy : 0.5
Iteration : 318 Cost : 0.693147 Accuracy : 0.5
Iteration : 319 Cost : 0.693147 Accuracy : 0.5
Iteration : 320 Cost : 0.693147 Accuracy : 0.5
Iteration : 321 Cost : 0.693147 Accuracy : 0.5
Iteration : 322 Cost : 0.693147 Accuracy : 0.5
Iteration : 323 Cost : 0.693147 Accuracy : 0.5
Iteration : 324 Cost : 0.693147 Accuracy : 0.5
Iteration : 325 Cost : 0.693147 Accuracy : 0.5
Iteration : 326 Cost : 0.693147 Accuracy : 0.5
Iteration : 327 Cost : 0.693147 Accuracy : 0.5
Iteration : 3

Iteration : 583 Cost : 0.693145 Accuracy : 0.5
Iteration : 584 Cost : 0.693145 Accuracy : 0.5
Iteration : 585 Cost : 0.693145 Accuracy : 0.5
Iteration : 586 Cost : 0.693145 Accuracy : 0.5
Iteration : 587 Cost : 0.693145 Accuracy : 0.5
Iteration : 588 Cost : 0.693145 Accuracy : 0.5
Iteration : 589 Cost : 0.693145 Accuracy : 0.5
Iteration : 590 Cost : 0.693145 Accuracy : 0.5
Iteration : 591 Cost : 0.693145 Accuracy : 0.5
Iteration : 592 Cost : 0.693145 Accuracy : 0.5
Iteration : 593 Cost : 0.693145 Accuracy : 0.5
Iteration : 594 Cost : 0.693145 Accuracy : 0.5
Iteration : 595 Cost : 0.693145 Accuracy : 0.5
Iteration : 596 Cost : 0.693145 Accuracy : 0.5
Iteration : 597 Cost : 0.693145 Accuracy : 0.5
Iteration : 598 Cost : 0.693145 Accuracy : 0.5
Iteration : 599 Cost : 0.693145 Accuracy : 0.5
Iteration : 600 Cost : 0.693145 Accuracy : 0.5
Iteration : 601 Cost : 0.693145 Accuracy : 0.5
Iteration : 602 Cost : 0.693145 Accuracy : 0.5
Iteration : 603 Cost : 0.693145 Accuracy : 0.5
Iteration : 6

Iteration : 827 Cost : 0.693143 Accuracy : 0.5
Iteration : 828 Cost : 0.693143 Accuracy : 0.5
Iteration : 829 Cost : 0.693143 Accuracy : 0.5
Iteration : 830 Cost : 0.693143 Accuracy : 0.5
Iteration : 831 Cost : 0.693143 Accuracy : 0.5
Iteration : 832 Cost : 0.693143 Accuracy : 0.5
Iteration : 833 Cost : 0.693143 Accuracy : 0.5
Iteration : 834 Cost : 0.693143 Accuracy : 0.5
Iteration : 835 Cost : 0.693143 Accuracy : 0.5
Iteration : 836 Cost : 0.693143 Accuracy : 0.5
Iteration : 837 Cost : 0.693143 Accuracy : 0.5
Iteration : 838 Cost : 0.693143 Accuracy : 0.5
Iteration : 839 Cost : 0.693143 Accuracy : 0.5
Iteration : 840 Cost : 0.693143 Accuracy : 0.5
Iteration : 841 Cost : 0.693143 Accuracy : 0.5
Iteration : 842 Cost : 0.693143 Accuracy : 0.5
Iteration : 843 Cost : 0.693143 Accuracy : 0.5
Iteration : 844 Cost : 0.693143 Accuracy : 0.5
Iteration : 845 Cost : 0.693143 Accuracy : 0.5
Iteration : 846 Cost : 0.693143 Accuracy : 0.5
Iteration : 847 Cost : 0.693143 Accuracy : 0.5
Iteration : 8

Iteration : 1080 Cost : 0.693138 Accuracy : 0.5
Iteration : 1081 Cost : 0.693138 Accuracy : 0.5
Iteration : 1082 Cost : 0.693138 Accuracy : 0.5
Iteration : 1083 Cost : 0.693138 Accuracy : 0.5
Iteration : 1084 Cost : 0.693138 Accuracy : 0.5
Iteration : 1085 Cost : 0.693138 Accuracy : 0.5
Iteration : 1086 Cost : 0.693138 Accuracy : 0.5
Iteration : 1087 Cost : 0.693138 Accuracy : 0.5
Iteration : 1088 Cost : 0.693138 Accuracy : 0.5
Iteration : 1089 Cost : 0.693138 Accuracy : 0.5
Iteration : 1090 Cost : 0.693138 Accuracy : 0.5
Iteration : 1091 Cost : 0.693138 Accuracy : 0.5
Iteration : 1092 Cost : 0.693138 Accuracy : 0.5
Iteration : 1093 Cost : 0.693138 Accuracy : 0.5
Iteration : 1094 Cost : 0.693138 Accuracy : 0.5
Iteration : 1095 Cost : 0.693138 Accuracy : 0.5
Iteration : 1096 Cost : 0.693138 Accuracy : 0.5
Iteration : 1097 Cost : 0.693138 Accuracy : 0.5
Iteration : 1098 Cost : 0.693138 Accuracy : 0.5
Iteration : 1099 Cost : 0.693138 Accuracy : 0.5
Iteration : 1100 Cost : 0.693138 Accurac

...

Iteration : 1509 Cost : 0.693104 Accuracy : 0.5
Iteration : 1510 Cost : 0.693104 Accuracy : 0.5
Iteration : 1511 Cost : 0.693103 Accuracy : 0.5
Iteration : 1512 Cost : 0.693103 Accuracy : 0.5
Iteration : 1513 Cost : 0.693103 Accuracy : 0.5
Iteration : 1514 Cost : 0.693103 Accuracy : 0.5
Iteration : 1515 Cost : 0.693103 Accuracy : 0.5
Iteration : 1516 Cost : 0.693102 Accuracy : 0.5
Iteration : 1517 Cost : 0.693102 Accuracy : 0.5
Iteration : 1518 Cost : 0.693102 Accuracy : 0.5
Iteration : 1519 Cost : 0.693102 Accuracy : 0.5
Iteration : 1520 Cost : 0.693101 Accuracy : 0.5
Iteration : 1521 Cost : 0.693101 Accuracy : 0.5
Iteration : 1522 Cost : 0.693101 Accuracy : 0.5
Iteration : 1523 Cost : 0.693101 Accuracy : 0.5
Iteration : 1524 Cost : 0.693101 Accuracy : 0.5
Iteration : 1525 Cost : 0.6931 Accuracy : 0.5
Iteration : 1526 Cost : 0.6931 Accuracy : 0.5
Iteration : 1527 Cost : 0.6931 Accuracy : 0.5
Iteration : 1528 Cost : 0.6931 Accuracy : 0.5
Iteration : 1529 Cost : 0.693099 Accuracy : 0.5


Iteration : 1721 Cost : 0.692987 Accuracy : 0.5
Iteration : 1722 Cost : 0.692986 Accuracy : 0.5
Iteration : 1723 Cost : 0.692984 Accuracy : 0.5
Iteration : 1724 Cost : 0.692983 Accuracy : 0.5
Iteration : 1725 Cost : 0.692982 Accuracy : 0.5
Iteration : 1726 Cost : 0.69298 Accuracy : 0.5
Iteration : 1727 Cost : 0.692979 Accuracy : 0.5
Iteration : 1728 Cost : 0.692978 Accuracy : 0.5
Iteration : 1729 Cost : 0.692976 Accuracy : 0.5
Iteration : 1730 Cost : 0.692975 Accuracy : 0.5
Iteration : 1731 Cost : 0.692973 Accuracy : 0.5
Iteration : 1732 Cost : 0.692972 Accuracy : 0.5
Iteration : 1733 Cost : 0.69297 Accuracy : 0.5
Iteration : 1734 Cost : 0.692969 Accuracy : 0.5
Iteration : 1735 Cost : 0.692967 Accuracy : 0.5
Iteration : 1736 Cost : 0.692966 Accuracy : 0.5
Iteration : 1737 Cost : 0.692964 Accuracy : 0.5
Iteration : 1738 Cost : 0.692963 Accuracy : 0.5
Iteration : 1739 Cost : 0.692961 Accuracy : 0.5
Iteration : 1740 Cost : 0.69296 Accuracy : 0.5
Iteration : 1741 Cost : 0.692958 Accuracy :

Iteration : 1963 Cost : 0.688474 Accuracy : 0.75
Iteration : 1964 Cost : 0.688365 Accuracy : 0.75
Iteration : 1965 Cost : 0.688252 Accuracy : 0.75
Iteration : 1966 Cost : 0.688136 Accuracy : 0.75
Iteration : 1967 Cost : 0.688017 Accuracy : 0.75
Iteration : 1968 Cost : 0.687895 Accuracy : 0.75
Iteration : 1969 Cost : 0.68777 Accuracy : 0.75
Iteration : 1970 Cost : 0.687641 Accuracy : 0.75
Iteration : 1971 Cost : 0.687509 Accuracy : 0.75
Iteration : 1972 Cost : 0.687373 Accuracy : 0.75
Iteration : 1973 Cost : 0.687234 Accuracy : 0.75
Iteration : 1974 Cost : 0.687091 Accuracy : 0.75
Iteration : 1975 Cost : 0.686944 Accuracy : 0.75
Iteration : 1976 Cost : 0.686794 Accuracy : 0.75
Iteration : 1977 Cost : 0.686639 Accuracy : 0.75
Iteration : 1978 Cost : 0.68648 Accuracy : 0.75
Iteration : 1979 Cost : 0.686318 Accuracy : 0.75
Iteration : 1980 Cost : 0.68615 Accuracy : 0.75
Iteration : 1981 Cost : 0.685979 Accuracy : 0.75
Iteration : 1982 Cost : 0.685803 Accuracy : 0.75
Iteration : 1983 Cost :

Iteration : 2131 Cost : 0.570165 Accuracy : 0.75
Iteration : 2132 Cost : 0.569125 Accuracy : 0.75
Iteration : 2133 Cost : 0.568091 Accuracy : 0.75
Iteration : 2134 Cost : 0.567064 Accuracy : 0.75
Iteration : 2135 Cost : 0.566042 Accuracy : 0.75
Iteration : 2136 Cost : 0.565028 Accuracy : 0.75
Iteration : 2137 Cost : 0.564019 Accuracy : 0.75
Iteration : 2138 Cost : 0.563017 Accuracy : 0.75
Iteration : 2139 Cost : 0.562021 Accuracy : 0.75
Iteration : 2140 Cost : 0.561031 Accuracy : 0.75
Iteration : 2141 Cost : 0.560047 Accuracy : 0.75
Iteration : 2142 Cost : 0.559069 Accuracy : 0.75
Iteration : 2143 Cost : 0.558097 Accuracy : 0.75
Iteration : 2144 Cost : 0.557131 Accuracy : 0.75
Iteration : 2145 Cost : 0.55617 Accuracy : 0.75
Iteration : 2146 Cost : 0.555216 Accuracy : 0.75
Iteration : 2147 Cost : 0.554266 Accuracy : 0.75
Iteration : 2148 Cost : 0.553322 Accuracy : 0.75
Iteration : 2149 Cost : 0.552383 Accuracy : 0.75
Iteration : 2150 Cost : 0.55145 Accuracy : 0.75
Iteration : 2151 Cost 

Iteration : 2365 Cost : 0.152998 Accuracy : 1.0
Iteration : 2366 Cost : 0.151454 Accuracy : 1.0
Iteration : 2367 Cost : 0.149932 Accuracy : 1.0
Iteration : 2368 Cost : 0.148434 Accuracy : 1.0
Iteration : 2369 Cost : 0.146957 Accuracy : 1.0
Iteration : 2370 Cost : 0.145503 Accuracy : 1.0
Iteration : 2371 Cost : 0.14407 Accuracy : 1.0
Iteration : 2372 Cost : 0.142658 Accuracy : 1.0
Iteration : 2373 Cost : 0.141267 Accuracy : 1.0

